In [1]:
import logging
from telegram.ext import Updater, MessageHandler, Filters
import IPython.display as ipd

import librosa
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor


In [2]:
with open("Telegram-bot-token.txt", "r") as f:
    TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists

HF_MODEL = "cantillation/whisper-medium-he-teamim-aviv-base"
model = WhisperForConditionalGeneration.from_pretrained(HF_MODEL).to("cuda")
processor = WhisperProcessor.from_pretrained(HF_MODEL, language="hebrew", task="transcribe")
SR = processor.feature_extractor.sampling_rate

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def extract_features(audio):
    feature = processor.feature_extractor(audio, sampling_rate=SR).input_features[0]
    return torch.tensor(feature).unsqueeze(0)


In [4]:
def transcribe(audio):

    # preprocess the audio file
    inputs = extract_features(audio).to("cuda")
    
    # generate the text
    generated_ids = model.generate(inputs, max_length=225, num_beams=4, early_stopping=True)
    transcription = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    return transcription


In [5]:
# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

# Define a function to handle audio messages
def handle_audio(update, context):
    audio_message = update.message.voice or update.message.audio
    # Get the audio file
    file = context.bot.get_file(audio_message.file_id)
    audio = file.download()
    audio = librosa.load(audio, sr=SR)[0]
    
    # Send a message to the user
    context.bot.send_message(chat_id=update.message.chat_id, text="קיבלתי את הקובץ, אני מתחיל להמיר אותו לטקסט עם טעמים...")
    
    # Audio to text with cantillations
    transcription = transcribe(audio)
    
    # Send the transcription to the user
    context.bot.send_message(chat_id=update.message.chat_id, text=f"זה מוכן!: \n {transcription}")
    
    # delete the audio file
    os.remove(audio)
    
    
def main():
    # Create an instance of the Updater class
    updater = Updater(TOKEN, use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Register a handler for audio messages
    audio_handler = MessageHandler(Filters.audio, handle_audio)
    dispatcher.add_handler(audio_handler)
    
    # Register a handler for voice messages
    voice_handler = MessageHandler(Filters.voice, handle_audio)
    dispatcher.add_handler(voice_handler)
    
    # Start the bot
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

2024-02-28 17:44:36,171 - apscheduler.scheduler - INFO - Scheduler started
2024-02-28 17:45:30.184845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 17:45:30.276530: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.2/lib64
2024-02-28 17:45:30.276545: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-02-28 17:45:30.761712: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dyna